In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import time

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
x_train = np.load('x_train.npy')
y_train = np.load('y_train.npy')
x_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')

In [3]:
num_classes = 10
n1 = 10
no_users = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [4]:
x_train = np.reshape(x_train, (x_train.shape[0], -1))/255
x_test = np.reshape(x_test, (x_test.shape[0], -1))/255

In [5]:
# Network parameters.
n_hidden_1 = 128 # 1st layer number of neurons.
n_hidden_2 = 64 # 2nd layer number of neurons.

In [6]:
class User:
    def __init__(self):
        self.weights = {
            'W1': tf.Variable(tf.random.truncated_normal([784,n_hidden_1], stddev=0.1)),
            'W2': tf.Variable(tf.random.truncated_normal([n_hidden_1,n_hidden_2], stddev=0.1)),
            'W3': tf.Variable(tf.random.truncated_normal([n_hidden_2,10], stddev=0.1)),
        }

        self.lambda_W1_l = tf.Variable(tf.zeros([784*n_hidden_1,1]))
        self.lambda_W2_l = tf.Variable(tf.zeros([n_hidden_1*n_hidden_2,1]))
        self.lambda_W3_l = tf.Variable(tf.zeros([n_hidden_2*10,1]))
        
        self.lambda_W1_r = tf.Variable(tf.zeros([784*n_hidden_1,1]))
        self.lambda_W2_r = tf.Variable(tf.zeros([n_hidden_1*n_hidden_2,1]))
        self.lambda_W3_r = tf.Variable(tf.zeros([n_hidden_2*10,1]))
        
        

    def neural_net(self, x):
        y1 = tf.nn.relu(tf.matmul(x, self.weights['W1']))
        y2 = tf.nn.relu(tf.matmul(y1, self.weights['W2']))
        ylogits = tf.matmul(y2, self.weights['W3'])
        return tf.nn.softmax(ylogits)

In [7]:
# mini-batch loss function.
def mini_batches(X, Y, mb_size = 100):

    m = X.shape[0]

    perm = list(np.random.permutation(m))
    #perm = perm_init[0:100]
    X_temp = X[perm,:]
    Y_temp = Y[perm,:].reshape((m, Y.shape[1]))
    
    X_r = tf.convert_to_tensor(X_temp[0:mb_size,:], dtype=np.float32)
    Y_r = tf.convert_to_tensor(Y_temp[0:mb_size,:], dtype=np.float32)
    return X_r,Y_r

In [8]:
# Cross-Entropy loss function.
def cross_entropy(y_pred, y_true):
    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    # Compute cross-entropy.
    return -tf.reduce_sum(y_true * tf.math.log(y_pred))

In [9]:
# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

In [10]:
# Optimization process. 
def run_optimization(x, y, user, user_l, user_r, rho, flag):
    # Wrap computation inside a GradientTape for automatic differentiation.
    #temp1_r = tf.reshape(user_r.weights['W1'],[784*n_hidden_1, 1])
    #temp2_r = tf.reshape(user_r.weights['W2'],[n_hidden_1*n_hidden_2, 1])
    #temp3_r = tf.reshape(user_r.weights['W3'],[10*n_hidden_2, 1])
    
    #temp1_l = tf.reshape(user_l.weights['W1'],[784*n_hidden_1, 1])
    #temp2_l = tf.reshape(user_l.weights['W2'],[n_hidden_1*n_hidden_2, 1])
    #temp3_l = tf.reshape(user_l.weights['W3'],[10*n_hidden_2, 1])
    
    with tf.GradientTape() as g:
        #g.watch(x)
        pred = user.neural_net(x)
        if (flag == 0):
            loss = cross_entropy(pred, y)\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W1'],[784*n_hidden_1, 1])-tf.reshape(user_r.weights['W1'],[784*n_hidden_1, 1])))\
            + tf.reduce_sum(tf.math.multiply(user.lambda_W1_r,tf.reshape(user.weights['W1'],[784*n_hidden_1, 1])-tf.reshape(user_r.weights['W1'],[784*n_hidden_1, 1])))\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W2'],[n_hidden_1*n_hidden_2, 1])-tf.reshape(user_r.weights['W2'],[n_hidden_1*n_hidden_2, 1]))) \
            + tf.reduce_sum(tf.math.multiply(user.lambda_W2_r,tf.reshape(user.weights['W2'],[n_hidden_1*n_hidden_2, 1])-tf.reshape(user_r.weights['W2'],[n_hidden_1*n_hidden_2, 1])))\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W3'],[10*n_hidden_2, 1])-tf.reshape(user_r.weights['W3'],[10*n_hidden_2, 1]))) \
            + tf.reduce_sum(tf.math.multiply(user.lambda_W3_r,tf.reshape(user.weights['W3'],[10*n_hidden_2, 1])-tf.reshape(user_r.weights['W3'],[10*n_hidden_2, 1])))

        elif (flag == 1):
            loss = cross_entropy(pred, y)\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W1'],[784*n_hidden_1, 1])-tf.reshape(user_l.weights['W1'],[784*n_hidden_1, 1])))\
            + tf.reduce_sum(tf.math.multiply(user.lambda_W1_l,-tf.reshape(user.weights['W1'],[784*n_hidden_1, 1])+tf.reshape(user_l.weights['W1'],[784*n_hidden_1, 1])))\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W2'],[n_hidden_1*n_hidden_2, 1])-tf.reshape(user_l.weights['W2'],[n_hidden_1*n_hidden_2, 1]))) \
            + tf.reduce_sum(tf.math.multiply(user.lambda_W2_l,-tf.reshape(user.weights['W2'],[n_hidden_1*n_hidden_2, 1])+tf.reshape(user_l.weights['W2'],[n_hidden_1*n_hidden_2, 1])))\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W3'],[10*n_hidden_2, 1])-tf.reshape(user_l.weights['W3'],[10*n_hidden_2, 1]))) \
            + tf.reduce_sum(tf.math.multiply(user.lambda_W3_l,-tf.reshape(user.weights['W3'],[10*n_hidden_2, 1])+tf.reshape(user_l.weights['W3'],[10*n_hidden_2, 1])))

        else:
            loss = cross_entropy(pred, y)\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W1'],[784*n_hidden_1, 1])-tf.reshape(user_r.weights['W1'],[784*n_hidden_1, 1])))\
            + tf.reduce_sum(tf.math.multiply(user.lambda_W1_r,tf.reshape(user.weights['W1'],[784*n_hidden_1, 1])-tf.reshape(user_r.weights['W1'],[784*n_hidden_1, 1])))\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W2'],[n_hidden_1*n_hidden_2, 1])-tf.reshape(user_r.weights['W2'],[n_hidden_1*n_hidden_2, 1]))) \
            + tf.reduce_sum(tf.math.multiply(user.lambda_W2_r,tf.reshape(user.weights['W2'],[n_hidden_1*n_hidden_2, 1])-tf.reshape(user_r.weights['W2'],[n_hidden_1*n_hidden_2, 1])))\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W3'],[10*n_hidden_2, 1])-tf.reshape(user_r.weights['W3'],[10*n_hidden_2, 1]))) \
            + tf.reduce_sum(tf.math.multiply(user.lambda_W3_r,tf.reshape(user.weights['W3'],[10*n_hidden_2, 1])-tf.reshape(user_r.weights['W3'],[10*n_hidden_2, 1])))
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W1'],[784*n_hidden_1, 1])-tf.reshape(user_l.weights['W1'],[784*n_hidden_1, 1])))\
            + tf.reduce_sum(tf.math.multiply(user.lambda_W1_l,-tf.reshape(user.weights['W1'],[784*n_hidden_1, 1])+tf.reshape(user_l.weights['W1'],[784*n_hidden_1, 1])))\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W2'],[n_hidden_1*n_hidden_2, 1])-tf.reshape(user_l.weights['W2'],[n_hidden_1*n_hidden_2, 1]))) \
            + tf.reduce_sum(tf.math.multiply(user.lambda_W2_l,-tf.reshape(user.weights['W2'],[n_hidden_1*n_hidden_2, 1])+tf.reshape(user_l.weights['W2'],[n_hidden_1*n_hidden_2, 1])))\
            + rho/2 * tf.reduce_sum(tf.math.square(tf.reshape(user.weights['W3'],[10*n_hidden_2, 1])-tf.reshape(user_l.weights['W3'],[10*n_hidden_2, 1]))) \
            + tf.reduce_sum(tf.math.multiply(user.lambda_W3_l,-tf.reshape(user.weights['W3'],[10*n_hidden_2, 1])+tf.reshape(user_l.weights['W3'],[10*n_hidden_2, 1])))

        
    
    # Variables to update, i.e. trainable variables.
    trainable_variables = user.weights.values()
    
    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)
    #print(gradients)
    
    # Update W following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    return cross_entropy(pred, y)

In [11]:
def find_max(WW1):
    
    myMax = (tf.reduce_max(WW1)).numpy()
    #myMax=R1_max.eval()
    
    myMin = (tf.reduce_min(WW1)).numpy()
    #myMin=R1_min.eval()
    
    return myMax, myMin

In [12]:
users = [User() for i in range(no_users)]
rho = 20
optimizer = tf.optimizers.Adam(learning_rate=0.001)
#optimizer = tf.optimizers.SGD(learning_rate=0.001)
#central_modal = [tf.Variable(tf.random.truncated_normal([784*n_hidden_1,1], stddev=0.1)), tf.Variable(tf.random.truncated_normal([n_hidden_1*n_hidden_2,1], stddev=0.1)), tf.Variable(tf.random.truncated_normal([10*n_hidden_2,1], stddev=0.1))]

In [13]:
x_train_k = []
y_train_k = []
data_per_worker = int(x_train.shape[0]/no_users)
for i in range(no_users):
    first = i*data_per_worker
    last = first + data_per_worker
    x_train_k.append(x_train[first:last])
    y_train_k.append(y_train[first:last])

In [14]:
x_test = tf.convert_to_tensor(x_test, dtype=np.float32)
y_test = tf.convert_to_tensor(y_test, dtype=np.float32)

In [15]:
R = tf.Variable(tf.zeros([1]))

In [16]:
import time

abs_weights_diff = []
abs_biases_diff = []
Train_Acc = []
Test_Acc = []
CrE_Train = []
CrE_Test = []


mb_size = 100
n_epochs = 400
b=8
tau=1/(2**b-1)

compTime = np.zeros([n_epochs,1])

acc_train = np.zeros([n_epochs,1])
acc_test = np.zeros([n_epochs,1])
acc_test_max = np.zeros([n_epochs,1])
total_loss = np.zeros([n_epochs,1])
acc_test_user0 = np.zeros([n_epochs,1])
alpha=0.01
ii = 0
localIter = 10
for k in range(n_epochs):
        print(k)
        if k > 0:
            compTime[k] = compTime[k-1]
        
        batch_x = []
        batch_y = []
        for i in range(no_users):
            batch_xx , batch_yy = mini_batches(x_train_k[i],y_train_k[i],  mb_size)
            batch_x.append(batch_xx)
            batch_y.append(batch_yy)
        
                
        # Run the optimization to update W  
        #for i in range(no_users):
        for i in range(0,no_users,2):
            if(i==0):
                flag = 0
                t = time.time()
                for kk in range(localIter):
                    loss = run_optimization(batch_x[i], batch_y[i], users[i], users[i], users[i+1], rho, flag)
                elapsed = time.time() - t
                compTime[k]=compTime[k]+elapsed
            elif (i == no_users-1):
                flag = 1
                for kk in range(localIter):
                    loss = run_optimization(batch_x[i], batch_y[i], users[i], users[i-1], users[i], rho, flag)
            else:
                flag = 2
                for kk in range(localIter):
                    loss = run_optimization(batch_x[i], batch_y[i], users[i], users[i-1], users[i+1], rho, flag)
                    
            total_loss[k] = total_loss[k] + loss
            
            W1 = tf.reshape(users[i].weights['W1'],[784*n_hidden_1, 1])
            W2 = tf.reshape(users[i].weights['W2'],[n_hidden_1*n_hidden_2, 1])
            W3 = tf.reshape(users[i].weights['W3'],[10*n_hidden_2, 1])
            
            if(i==0):
                t = time.time()
            
            [max_value, min_value]=find_max(W1)
            R[0].assign(max(abs(min_value), abs(max_value)))
            Q1=(W1+R[0])/(2*tau*R[0])+1/2
            Q1=tf.math.floor(Q1)#.eval())
            Q1= 2*tau*Q1*R[0]-R[0]
            users[i].weights['W1'].assign(tf.reshape(Q1,[784,n_hidden_1]))
            #sess.run(tf.assign(W11,Q11))
            #print(W11[0,0:5].eval())
            
            [max_value, min_value]=find_max(W2)
            R[0].assign(max(abs(min_value), abs(max_value)))
            Q2=(W2+R[0])/(2*tau*R[0])+1/2
            Q2=tf.math.floor(Q2)#.eval())
            Q2= 2*tau*Q2*R[0]-R[0]
            users[i].weights['W2'].assign(tf.reshape(Q2,[n_hidden_1,n_hidden_2]))
            
            [max_value, min_value]=find_max(W3)
            R[0].assign(max(abs(min_value), abs(max_value)))
            Q3=(W3+R[0])/(2*tau*R[0])+1/2
            Q3=tf.math.floor(Q3)#.eval())
            Q3= 2*tau*Q3*R[0]-R[0]
            users[i].weights['W3'].assign(tf.reshape(Q3,[n_hidden_2, 10]))
            
            if(i==0):
                elapsed = time.time() - t
                compTime[k]=compTime[k]+elapsed
            
        for i in range(1,no_users,2):
            if(i==0):
                flag = 0
                for kk in range(localIter):
                    loss = run_optimization(batch_x[i], batch_y[i], users[i], users[i], users[i+1], rho, flag)
            elif (i == no_users-1):
                flag = 1
                for kk in range(localIter):
                    loss = run_optimization(batch_x[i], batch_y[i], users[i], users[i-1], users[i], rho, flag)
            else:
                flag = 2
                for kk in range(localIter):
                    loss = run_optimization(batch_x[i], batch_y[i], users[i], users[i-1], users[i+1], rho, flag)
                    
            W1 = tf.reshape(users[i].weights['W1'],[784*n_hidden_1, 1])
            W2 = tf.reshape(users[i].weights['W2'],[n_hidden_1*n_hidden_2, 1])
            W3 = tf.reshape(users[i].weights['W3'],[10*n_hidden_2, 1])
            
            [max_value, min_value]=find_max(W1)
            R[0].assign(max(abs(min_value), abs(max_value)))
            Q1=(W1+R[0])/(2*tau*R[0])+1/2
            Q1=tf.math.floor(Q1)#.eval())
            Q1= 2*tau*Q1*R[0]-R[0]
            users[i].weights['W1'].assign(tf.reshape(Q1,[784,n_hidden_1]))
            #sess.run(tf.assign(W11,Q11))
            #print(W11[0,0:5].eval())
            
            [max_value, min_value]=find_max(W2)
            R[0].assign(max(abs(min_value), abs(max_value)))
            Q2=(W2+R[0])/(2*tau*R[0])+1/2
            Q2=tf.math.floor(Q2)#.eval())
            Q2= 2*tau*Q2*R[0]-R[0]
            users[i].weights['W2'].assign(tf.reshape(Q2,[n_hidden_1,n_hidden_2]))
            
            [max_value, min_value]=find_max(W3)
            R[0].assign(max(abs(min_value), abs(max_value)))
            Q3=(W3+R[0])/(2*tau*R[0])+1/2
            Q3=tf.math.floor(Q3)#.eval())
            Q3= 2*tau*Q3*R[0]-R[0]
            users[i].weights['W3'].assign(tf.reshape(Q3,[n_hidden_2, 10]))
            
            total_loss[k] = total_loss[k] + loss
        
        total_loss[k] = total_loss[k]/no_users
        print(total_loss[k])
        
        primal_residual=0
        for i in range(no_users):
            W1 = tf.reshape(users[i].weights['W1'],[784*n_hidden_1, 1])
            W2 = tf.reshape(users[i].weights['W2'],[n_hidden_1*n_hidden_2, 1])
            W3 = tf.reshape(users[i].weights['W3'],[10*n_hidden_2, 1])
            if (i==0):
                W1_r = tf.reshape(users[i+1].weights['W1'],[784*n_hidden_1, 1])
                W2_r = tf.reshape(users[i+1].weights['W2'],[n_hidden_1*n_hidden_2, 1])
                W3_r = tf.reshape(users[i+1].weights['W3'],[10*n_hidden_2, 1])
                
                yy1 = (users[i].lambda_W1_r + alpha*rho*(W1 - W1_r)).numpy()
                yy2 = (users[i].lambda_W2_r + alpha*rho*(W2 - W2_r)).numpy()
                yy3 = (users[i].lambda_W3_r + alpha*rho*(W3 - W3_r)).numpy()
                
                users[i].lambda_W1_r.assign(yy1) 
                users[i].lambda_W2_r.assign(yy2) 
                users[i].lambda_W3_r.assign(yy3)
                
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W1 - W1_r)).numpy()
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W2 - W2_r)).numpy()
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W3 - W3_r)).numpy()
            
            elif (i==no_users-1):
                W1_l = tf.reshape(users[i-1].weights['W1'],[784*n_hidden_1, 1])
                W2_l = tf.reshape(users[i-1].weights['W2'],[n_hidden_1*n_hidden_2, 1])
                W3_l = tf.reshape(users[i-1].weights['W3'],[10*n_hidden_2, 1])
                
                yy1 = (users[i].lambda_W1_l - alpha*rho*(W1 - W1_l)).numpy()
                yy2 = (users[i].lambda_W2_l - alpha*rho*(W2 - W2_l)).numpy()
                yy3 = (users[i].lambda_W3_l - alpha*rho*(W3 - W3_l)).numpy()
                
                users[i].lambda_W1_l.assign(yy1) 
                users[i].lambda_W2_l.assign(yy2) 
                users[i].lambda_W3_l.assign(yy3)
                
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W1 - W1_l)).numpy()
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W2 - W2_l)).numpy()
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W3 - W3_l)).numpy()
            
            else:
                W1_r = tf.reshape(users[i+1].weights['W1'],[784*n_hidden_1, 1])
                W2_r = tf.reshape(users[i+1].weights['W2'],[n_hidden_1*n_hidden_2, 1])
                W3_r = tf.reshape(users[i+1].weights['W3'],[10*n_hidden_2, 1])
                
                W1_l = tf.reshape(users[i-1].weights['W1'],[784*n_hidden_1, 1])
                W2_l = tf.reshape(users[i-1].weights['W2'],[n_hidden_1*n_hidden_2, 1])
                W3_l = tf.reshape(users[i-1].weights['W3'],[10*n_hidden_2, 1])
                
                
                yy1 = (users[i].lambda_W1_r + alpha*rho*(W1 - W1_r)).numpy()
                yy2 = (users[i].lambda_W2_r + alpha*rho*(W2 - W2_r)).numpy()
                yy3 = (users[i].lambda_W3_r + alpha*rho*(W3 - W3_r)).numpy()
                
                yy4 = (users[i].lambda_W1_l - alpha*rho*(W1 - W1_l)).numpy()
                yy5 = (users[i].lambda_W2_l - alpha*rho*(W2 - W2_l)).numpy()
                yy6 = (users[i].lambda_W3_l - alpha*rho*(W3 - W3_l)).numpy()
                
                users[i].lambda_W1_r.assign(yy1) 
                users[i].lambda_W2_r.assign(yy2) 
                users[i].lambda_W3_r.assign(yy3)
            
                users[i].lambda_W1_l.assign(yy4) 
                users[i].lambda_W2_l.assign(yy5) 
                users[i].lambda_W3_l.assign(yy6)
                
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W1 - W1_r)).numpy()
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W2 - W2_r)).numpy()
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W3 - W3_r)).numpy()
                
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W1 - W1_l)).numpy()
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W2 - W2_l)).numpy()
                primal_residual = primal_residual +tf.reduce_sum(tf.math.square(W3 - W3_l)).numpy()
                
            
            #users[i].lambda_W1.assign(yy1) 
            #users[i].lambda_W2.assign(yy2) 
            #users[i].lambda_W3.assign(yy3)
            
        print(primal_residual)
        
        
        train_acc = []
        test_acc = []
        for j in range(no_users):  
            train_pred = users[j].neural_net(batch_x[j])
            train_acc.append(accuracy(train_pred , batch_y[j]))
            test_pred = users[j].neural_net(x_test)
            test_acc.append(accuracy(test_pred , y_test))
            

        
        avgAcc_Train = np.mean(train_acc)
        avgAcc_Test = np.mean(test_acc)
        max_Test = np.max(test_acc)
        print(avgAcc_Train)
        print(avgAcc_Test)
        print(max_Test)
        print(compTime[k])
        acc_train[k] = avgAcc_Train
        acc_test[k] = avgAcc_Test
        acc_test_max[k] = max_Test
            
        #user0.weights['W1'] = tf.reshape(central_modal[0],[784,n_hidden_1])
        #user0.weights['W2'] = tf.reshape(central_modal[1],[n_hidden_1,n_hidden_2])
        #user0.weights['W3'] = tf.reshape(central_modal[2],[n_hidden_2, 10])
        
        #test_pred = user0.neural_net(x_test)
        #test_acc_user0 = accuracy(test_pred, y_test).numpy()
        #acc_test_user0[k]=test_acc_user0
        #print(test_acc_user0)
            

#np.save('loss_analog_4096_3.npy', total_loss)

#np.save('acc_test_digital_4096_user0_3.npy', acc_test_user0)

#np.save('loss_digital_40960_5.npy', total_loss)
#np.save('acc_train_digital_40960_5.npy', acc_train)
#np.save('acc_test_digital_40960_5.npy', acc_test)

0
[219.52255859]
23571.06405353546
0.287
0.19223
0.2524
[0.23450494]
1
[207.6260498]
14408.436212539673
0.47399998
0.33499998
0.5185
[0.4309268]
2
[195.38721924]
8345.580350875854
0.47899994
0.35776
0.4628
[0.61863875]
3
[179.09669189]
4982.5600028038025
0.541
0.43283
0.6865
[0.83598971]
4
[148.98980713]
3208.7516672611237
0.665
0.54093
0.7799
[1.02680564]
5
[115.57480469]
2278.267831325531
0.761
0.64676
0.825
[1.2153275]
6
[85.78131104]
1767.6086280345917
0.80600005
0.68468004
0.8396
[1.47181058]
7
[66.60859985]
1457.4669824838638
0.8299999
0.70825
0.8409
[1.67659378]
8
[53.52459717]
1248.2270487248898
0.87700003
0.76365
0.8613
[1.92520475]
9
[43.38453064]
1091.7850576639175
0.904
0.80149
0.8637
[2.10340476]
10
[37.29289551]
960.885446280241
0.9129999
0.80196
0.8666
[2.31075382]
11
[35.39457703]
850.3172545880079
0.926
0.82361
0.861
[2.58565903]
12
[30.67279053]
752.3831006884575
0.93600005
0.82442
0.8685
[2.82778096]
13
[28.71786194]
665.8057970255613
0.95
0.84505004
0.8736
[3.245231

0.996
0.93414
0.9445
[24.82282114]
114
[4.01756554]
63.65888096392155
0.99700004
0.93542004
0.9411
[25.00967526]
115
[4.84866028]
62.76535366475582
0.99300003
0.93472993
0.9406
[25.20796657]
116
[4.26125336]
64.80426508188248
0.9940001
0.93360996
0.9451
[25.39964747]
117
[5.11491776]
66.15341572463512
0.99300003
0.9327499
0.9448
[25.59824061]
118
[3.28906441]
64.27193331345916
0.996
0.93529
0.9435
[25.79006267]
119
[3.79495506]
62.70333870127797
0.99700004
0.93391
0.945
[25.98713827]
120
[3.28613243]
63.87789177149534
0.99700004
0.9336699
0.946
[26.17896342]
121
[2.75233383]
61.251089833676815
0.99899995
0.93697995
0.9467
[26.37331533]
122
[3.54876289]
59.56843316182494
0.996
0.93856
0.9449
[26.5667522]
123
[4.38312378]
62.09579027444124
0.99700004
0.93578994
0.9452
[26.76532125]
124
[4.40478973]
63.547638066112995
0.99700004
0.93325007
0.9428
[26.95821953]
125
[5.5037056]
63.66387705504894
0.9940001
0.93332005
0.9436
[27.14570475]
126
[1.81635876]
60.669884011149406
0.99899995
0.93865

[2.28673878]
68.83941353112459
0.99700004
0.9404599
0.9499
[47.63852525]
224
[2.21944408]
69.19877966493368
0.995
0.94405
0.9512
[47.8193543]
225
[1.53784409]
69.3826295286417
0.99899995
0.941
0.9518
[48.00621223]
226
[2.1629837]
70.26566782966256
0.99700004
0.9413699
0.9496
[48.1905005]
227
[2.70020332]
70.2750398516655
0.996
0.94088995
0.9525
[48.37347531]
228
[2.11513443]
71.01659245043993
0.99700004
0.93884003
0.9518
[48.54850125]
229
[1.71814232]
67.59820935875177
0.996
0.94235003
0.9538
[48.72979403]
230
[1.47503033]
66.34076088666916
0.99899995
0.9476601
0.9545
[48.90827727]
231
[2.18418484]
70.60741148144007
0.99899995
0.9434899
0.9527
[49.08555317]
232
[1.96388626]
69.12420512363315
0.99700004
0.93979996
0.9517
[49.27158928]
233
[2.79700966]
67.7627430241555
0.9940001
0.94095993
0.9526
[49.47501922]
234
[2.4335371]
68.00446136295795
0.99799997
0.94342995
0.9555
[49.65859509]
235
[1.41605768]
69.53502583131194
0.99899995
0.9416801
0.9521
[49.83329296]
236
[3.61607933]
67.817638

[1.06975527]
77.88649878185242
0.99899995
0.9417
0.9587
[68.38751626]
335
[1.56162653]
78.18347378261387
0.996
0.94232
0.9572
[68.58368325]
336
[1.57035151]
78.62685180734843
0.99899995
0.94157
0.9572
[68.76623821]
337
[1.7442915]
79.7797506339848
0.99799997
0.93587
0.9561
[68.95518827]
338
[5.11921082]
77.8659190563485
0.996
0.9407999
0.9554
[69.22858214]
339
[1.88166904]
75.26679006963968
0.995
0.94349
0.9554
[69.41428614]
340
[1.57275944]
74.06068528816104
0.99700004
0.94255
0.956
[69.60287213]
341
[0.8718401]
71.89432076178491
1.0
0.94772995
0.9566
[69.7804184]
342
[1.16742325]
67.79490718245506
0.99899995
0.94892997
0.9577
[69.96539235]
343
[2.47658443]
68.07389525137842
0.99700004
0.94707
0.9588
[70.14866233]
344
[1.97640095]
70.94896987639368
0.995
0.94033
0.9586
[70.32538438]
345
[0.98200903]
70.52221005316824
0.99799997
0.93991005
0.958
[70.50797725]
346
[2.40129738]
72.07002159114927
0.99799997
0.94219
0.9585
[70.69244528]
347
[1.68018551]
70.02152199670672
0.99899995
0.94686

In [17]:
#np.save('qgadmm_dnn_train_5bits.npy', acc_train)
#np.save('qgadmm_dnn_mean_test_5bits.npy', acc_test)
np.save('qgadmm_dnn_max_test.npy', acc_test)
#np.save('qgadmm_dnn_compTime.npy', compTime)